![EvalML Logo](images/evalml_logo.png)

# What is EvalML
EvalML is an AutoML library to build optimized machine learning pipelines for domain-specific objective functions.


Combined with `Featuretools <www.featuretools.com>`__ and `Compose <compose.ml>`__, EvalML can be used to create a state-of-the-art end-to-to end machine learning solutions.


# Quick Start Example

In [9]:
import evalml

## Load Data

First, we load in the features and targets we want to use to train our model

In [2]:
X, y = evalml.demos.load_breast_cancer()

## Configure search

EvalML has many options to configure the pipeline search. At the minimum, we need to define an objective function. For simplicity, we will use the F1 score in this example as the objective function. However, the real power of EvalML is in using [domain-specific]() objective functions or [building your own](). 

In [3]:
clf = evalml.AutoClassifier(objective="f1",
                            max_pipelines=5)

In order to validate the results of the pipeline creation and optimization process, we will save some of our data as a holdout set

In [4]:
X_train, X_holdout, y_train, y_holdout = evalml.preprocessing.split_data(X, y, test_size=.2)

When we call `.fit()`, the search for the best pipeline will begin. 

In [5]:
clf.fit(X_train, y_train)

*****************************
* Beginning pipeline search *
*****************************

Optimizing for F1. Greater score is better.

Searching up to 5 pipelines. No time limit is set. Set one using max_time parameter.

Possible model types: random_forest, xgboost, linear_model

Testing LogisticRegression w/ imputation + scaling: 100%|██████████| 5/5 [00:10<00:00,  2.16s/it]

✔ Optimization finished


## See Pipeline Rankings

After the search is finished we can view all of the pipelines searched, ranked by score. Internally, EvalML performs cross validation to score the pipelines. If it notices, a high variance across cross validation folds, it will warn you.

In [6]:
clf.rankings

,id,pipeline_name,score,high_variance_cv,parameters
0,1,LogisticRegressionPipeline,0.941799,False,"{'penalty': 'l2', 'C': 8.444214828324364, 'imp..."
1,2,LogisticRegressionPipeline,0.941799,False,"{'penalty': 'l2', 'C': 6.239401330891865, 'imp..."
2,3,LogisticRegressionPipeline,0.941799,False,"{'penalty': 'l2', 'C': 0.5765626434012575, 'im..."
3,4,LogisticRegressionPipeline,0.931937,False,"{'penalty': 'l2', 'C': 8.123565600467177, 'imp..."
4,0,RFPipeline,0.890110,False,"{'n_estimators': 569, 'max_depth': 630, 'imput..."


## Describe pipeline

If we are interested in see more details about the pipeline we can describe it using the `id` from the rankings table

In [7]:
clf.describe_pipeline(3)

************************
* Pipeline Description *
************************

Pipeline Name: LogisticRegression w/ imputation + scaling
Model type: linear_model
Objective: F1 (greater is better)
Total training time (including CV): 0.1 seconds

Parameters
• penalty: l2
• C: 0.5765626434012575
• impute_strategy: mean

Cross Validation
        F1  Precision  Recall   AUC  Log Loss
0    0.948      0.938   0.948 0.926     2.272
1    0.959      0.939   0.959 0.937     1.818
2    0.942      0.947   0.942 0.924     2.516
mean 0.949      0.941   0.949 0.929     2.202
std  0.009      0.005   0.009 0.007     0.354


## Select Best pipeline
We can now select best pipeline and score it on our holdout data

In [8]:
pipeline = clf.best_pipeline
pipeline.score(X_holdout, y_holdout)

0.9600000000000001

# Getting Started
[What is EvalML](self)

[Install](install)

# Overview
[Setting up pipeline search](overview/pipeline_search)

[Exploring search results](overview/search_results)

[Objective  Functions](overview/objective_functions)

# Demos

[Fraud Prediction](demos/fraud)

[Lead Scoring](demos/lead_scoring)

# Resources
[API Reference](api_reference)